## Import Libraries

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader, TensorDataset, random_split

## Data Preparation

In [2]:
df=pd.read_csv("housing.csv")
df.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity'],
      dtype='object')

In [4]:
df.replace([np.inf, -np.inf], np.nan)
df.dropna(inplace=True) 

In [5]:
targets = df[["median_house_value"]].to_numpy()
inputs = df.drop(["median_house_value", 'ocean_proximity'], axis=1).to_numpy()  

In [6]:
inputs = torch.from_numpy(inputs)
inputs = inputs.float()
targets = torch.from_numpy(targets)
targets = targets.float()

In [7]:
inputs_mean = torch.mean(inputs, axis=0)
targets_mean = torch.mean(targets, axis=0)
inputs = inputs / inputs_mean  
targets = targets / targets_mean 

In [8]:
dataset = TensorDataset(inputs, targets)

In [9]:
train_loader = DataLoader(dataset, batch_size=64, shuffle=True)

## Define Two Models

In [11]:
model = nn.Sequential(nn.Linear(8, 4),
                      nn.ReLU(),

                      nn.Linear(4, 2),
                      nn.Softmax(),

                      
                      nn.Linear(2, 1),
                      nn.ReLU())
model_with_optim = nn.Sequential(nn.Linear(8, 4),
                      nn.Dropout(p=0.2),

                      nn.Linear(4, 2),
                      nn.Softmax(),

                      
                      nn.Linear(2, 1),
                      nn.ReLU())



In [12]:
# Utility function to train the model
def fit(num_epochs, model, criterion, optimizer, train_loader):
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        # Train with batches of data
        for xb,yb in train_loader:
            # 1. Generate predictions
            pred = model(xb)
            # 2. Calculate loss
            loss = criterion(pred, yb)
            # 3. Compute gradients
            loss.backward()
            # 4. Update parameters using gradients
            optimizer.step()
            # 5. Reset the gradients to zero
            optimizer.zero_grad()
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

## Fit Model

In [13]:
# Define loss function
criterion = F.mse_loss
# Define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
fit(200, model, criterion, optimizer, train_loader)

C:\Users\user\Anaconda3\lib\site-packages\torch\nn\modules\container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch [10/200], Loss: 0.2764
Epoch [20/200], Loss: 0.4241
Epoch [30/200], Loss: 0.3988
Epoch [40/200], Loss: 0.4176
Epoch [50/200], Loss: 0.4812
Epoch [60/200], Loss: 0.4477
Epoch [70/200], Loss: 0.4457
Epoch [80/200], Loss: 0.2092
Epoch [90/200], Loss: 0.2650
Epoch [100/200], Loss: 0.4924
Epoch [110/200], Loss: 0.1497
Epoch [120/200], Loss: 0.3648
Epoch [130/200], Loss: 0.2818
Epoch [140/200], Loss: 0.2573
Epoch [150/200], Loss: 0.4728
Epoch [160/200], Loss: 0.2407
Epoch [170/200], Loss: 0.3208
Epoch [180/200], Loss: 0.1944
Epoch [190/200], Loss: 0.2216
Epoch [200/200], Loss: 0.2620


## Fit Model with Regularization

In [15]:
# Define loss function
criterion = F.mse_loss
# Define optimizer
optimizer = torch.optim.SGD(model.parameters(),weight_decay=2, lr=1e-4)
fit(200, model, criterion, optimizer, train_loader)

Epoch [10/200], Loss: 0.5039
Epoch [20/200], Loss: 0.5231
Epoch [30/200], Loss: 0.6730
Epoch [40/200], Loss: 0.6671
Epoch [50/200], Loss: 0.3414
Epoch [60/200], Loss: 0.6801
Epoch [70/200], Loss: 0.3169
Epoch [80/200], Loss: 0.4528
Epoch [90/200], Loss: 0.8990
Epoch [100/200], Loss: 0.3164
Epoch [110/200], Loss: 0.3380
Epoch [120/200], Loss: 0.5582
Epoch [130/200], Loss: 0.2836
Epoch [140/200], Loss: 0.5343
Epoch [150/200], Loss: 0.6177
Epoch [160/200], Loss: 0.8089
Epoch [170/200], Loss: 0.6143
Epoch [180/200], Loss: 0.5407
Epoch [190/200], Loss: 1.3284
Epoch [200/200], Loss: 0.4217


## Fit Model with Regularization and dropout

In [16]:
# Define loss function
criterion = F.mse_loss
# Define optimizer
optimizer = torch.optim.SGD(model_with_optim.parameters(), lr=1e-4)
fit(200, model_with_optim, criterion, optimizer, train_loader)

Epoch [10/200], Loss: 1.0079
Epoch [20/200], Loss: 1.0350
Epoch [30/200], Loss: 1.6902
Epoch [40/200], Loss: 1.0565
Epoch [50/200], Loss: 1.5580
Epoch [60/200], Loss: 1.3978
Epoch [70/200], Loss: 0.8960
Epoch [80/200], Loss: 0.4581
Epoch [90/200], Loss: 1.2524
Epoch [100/200], Loss: 1.2755
Epoch [110/200], Loss: 0.9560
Epoch [120/200], Loss: 1.7975
Epoch [130/200], Loss: 1.4891
Epoch [140/200], Loss: 1.2462
Epoch [150/200], Loss: 1.9253
Epoch [160/200], Loss: 1.5834
Epoch [170/200], Loss: 0.8336
Epoch [180/200], Loss: 1.1612
Epoch [190/200], Loss: 0.9644
Epoch [200/200], Loss: 1.4126
